In [96]:
# ==========================================================
# EXTRACCIÓN DE CARACTERÍSTICAS PARA PREDICCIÓN DE HIPOXIA
# ¿Qué se hace en extracción de características?
#Variables originales:
#oxigeno_disuelto_mg_l, temperatura_c, ph, amoniaco_mg_l, nitrato_ppm, turbidez_ntu, manganeso_mg_l.
##################Nuevas variables derivadas (ejemplos):
#Promedio, mínimo, máximo y desviación en 1 hora.
#Pendiente (tasa de cambio).
#Diferencia simple con el dato anterior.
#Suavizado (EWMA) y residuo (cuánto se desvía DO de su tendencia).
#Interacciones fisicoquímicas: temp × DO, ph × amoníaco.
#Hora del día en seno/coseno (hipoxia ocurre más de madrugada).

In [32]:
# === 0) Librerías ===
import pandas as pd
import numpy as np
import pandas as pd


In [33]:
# === 1) Cargar dataset limpio SIN etiquetas ===
RUTA = "IoT_AquaSensors_Limpio_SinEtiquetas.csv"
df = pd.read_csv(RUTA, parse_dates=["marca_tiempo"])
df = df.sort_values(["estacion", "marca_tiempo"]).reset_index(drop=True)


In [34]:
# === 2) Crear etiqueta 'hipoxia' (y) desde oxígeno disuelto ===
UMBRAL_HIPOXIA = 3.0  # mg/L (ajústalo si tu docente indica otro)
df["hipoxia"] = (df["oxigeno_disuelto_mg_l"] < UMBRAL_HIPOXIA).astype(int)



In [36]:
# === 3) Definir sensores predictores (sin oxígeno directo) ===
sensores = ["temperatura_c", "ph", "amoniaco_mg_l",
            "nitrato_ppm", "turbidez_ntu", "manganeso_mg_l"]

In [37]:
# === 4) Función: extraer características por estación (usa SOLO pasado) ===
def extraer_caracteristicas_por_estacion(tab: pd.DataFrame) -> pd.DataFrame:
    # Orden temporal y usar tiempo como índice
    tab = tab.sort_values("marca_tiempo").set_index("marca_tiempo")
    ventana = "60min"  # 1 hora
    feats = {}

In [ ]:
# === PASO 4.1: Estadísticos por ventana (solo pasado) ===
import pandas as pd

sensores = ["temperatura_c","ph","amoniaco_mg_l","nitrato_ppm","turbidez_ntu","manganeso_mg_l"]

def paso_41_estadisticos(tab: pd.DataFrame) -> pd.DataFrame:
    tab = tab.sort_values("marca_tiempo").set_index("marca_tiempo")

    ventana = "60min"                     # ventana de 1 hora
    out = pd.DataFrame(index=tab.index)   # aquí guardamos las features

    # Estadísticos por ventana usando SOLO pasado (excluye instante actual)
    for col in sensores:
        r = tab[col].rolling(ventana, closed="left", min_periods=2)
        out[f"{col}_media_1h"] = r.mean()
        out[f"{col}_min_1h"]   = r.min()
        out[f"{col}_max_1h"]   = r.max()
        out[f"{col}_std_1h"]   = r.std()

    # Agregar claves y (si existe) etiqueta
    out = out.reset_index()
    out["estacion"] = tab["estacion"].values
    if "hipoxia" in tab.columns:
        out["hipoxia"] = tab["hipoxia"].values
    return out

# Aplicar por estación y unir
caracteristicas_41 = (
    df[["estacion","marca_tiempo"] + sensores + (["hipoxia"] if "hipoxia" in df.columns else [])]
      .groupby("estacion", group_keys=False)
      .apply(paso_41_estadisticos)
      .reset_index(drop=True)
)

# Quitar filas sin suficiente historial (NaN por ventana)
cols_feat_41 = [c for c in caracteristicas_41.columns if c not in ["estacion","marca_tiempo","hipoxia"]]
caracteristicas_41 = caracteristicas_41.dropna(subset=cols_feat_41).reset_index(drop=True)

print("OK 4.1 ->", caracteristicas_41.shape)
print(caracteristicas_41.head(3))

In [39]:
# === PASO 4.2: Tendencias previas de oxígeno + unir con 4.1 y guardar ===
import numpy as np
import pandas as pd

# Si aún no creaste la etiqueta, créala (no afecta si ya existe)
if "hipoxia" not in df.columns:
    df["hipoxia"] = (df["oxigeno_disuelto_mg_l"] < 3.0).astype(int)

# ---- Función de tendencias previas por estación ----
def paso_42_tendencias(tab: pd.DataFrame) -> pd.DataFrame:
    tab = tab.sort_values("marca_tiempo").set_index("marca_tiempo")

    ox   = tab["oxigeno_disuelto_mg_l"]
    dif  = ox.diff()
    dt_h = tab.index.to_series().diff().dt.total_seconds() / 3600.0

    out = pd.DataFrame(index=tab.index)
    out["oxigeno_dif_1_prev"]       = dif.shift(1)                 # cambio previo
    out["oxigeno_pendiente_h_prev"] = (dif / dt_h).shift(1)        # pendiente previa
    out["oxigeno_ewma_1h_prev"]     = ox.ewm(span=3, adjust=False).mean().shift(1)  # suavizado previo

    # Señales de hora del día (ciclo)
    hora = tab.index.hour + tab.index.minute/60.0
    out["hora_sin"] = np.sin(2*np.pi*hora/24)
    out["hora_cos"] = np.cos(2*np.pi*hora/24)

    out = out.reset_index()
    out["estacion"] = tab["estacion"].values
    out["hipoxia"]  = tab["hipoxia"].values
    return out

# ---- Aplicar 4.2 por estación ----
caracteristicas_42 = (
    df[["estacion","marca_tiempo","oxigeno_disuelto_mg_l","hipoxia"]]
      .groupby("estacion", group_keys=False)
      .apply(paso_42_tendencias)
      .reset_index(drop=True)
)

# ---- Unir con el resultado de 4.1 (caracteristicas_41) ----
caracteristicas = caracteristicas_41.merge(
    caracteristicas_42,
    on=["estacion","marca_tiempo","hipoxia"],  # si no tuvieras hipoxia en 4.1, usa solo ['estacion','marca_tiempo']
    how="inner"
)

# Si te aparecen columnas duplicadas de hipoxia (hipoxia_x/hipoxia_y):
if "hipoxia_x" in caracteristicas.columns:
    caracteristicas = (caracteristicas
                       .rename(columns={"hipoxia_x":"hipoxia"})
                       .drop(columns=[c for c in ["hipoxia_y"] if c in caracteristicas.columns]))

# ---- Quitar filas sin historial suficiente (NaN) ----
cols_feat = [c for c in caracteristicas.columns if c not in ["estacion","marca_tiempo","hipoxia"]]
caracteristicas = caracteristicas.dropna(subset=cols_feat).reset_index(drop=True)

# ---- Guardar ----
caracteristicas.to_csv("caracteristicas.csv", index=False)
print("Guardado: caracteristicas.csv")
print("Shape:", caracteristicas.shape)
print("Distribución hipoxia:\n", caracteristicas["hipoxia"].value_counts())
print("Columnas (muestra):", caracteristicas.columns[:12].tolist())

#Calcula oxigeno_dif_1_prev, oxigeno_pendiente_h_prev, oxigeno_ewma_1h_prev (solo pasado).
#Añade hora_sin y hora_cos.
#Une con las estadísticas 1h del PASO 4.1 (caracteristicas_41).
#Elimina filas con NaN de ventanas/diffs.
#Guarda caracteristicas.csv listo para modelar.

/tmp/ipykernel_793/2477385284.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(paso_42_tendencias)


Guardado: caracteristicas.csv
Shape: (74727, 32)
Distribución hipoxia:
 hipoxia
0    72503
1     2224
Name: count, dtype: int64
Columnas (muestra): ['marca_tiempo', 'temperatura_c_media_1h', 'temperatura_c_min_1h', 'temperatura_c_max_1h', 'temperatura_c_std_1h', 'ph_media_1h', 'ph_min_1h', 'ph_max_1h', 'ph_std_1h', 'amoniaco_mg_l_media_1h', 'amoniaco_mg_l_min_1h', 'amoniaco_mg_l_max_1h']
